In [1]:
# Import libraries
import requests, json, os, re, configparser
import openai

#from google.cloud import bigquery
#from google.oauth2 import service_account
#import pandas_gbq

#config = configparser.ConfigParser()
#config.read('keys/config.ini')

In [2]:
# Set environment variables
# Google Cloud
key_path = 'keys/appgpt-374716-6184be28027d.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'keys/appgpt_bigquery.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
pandas_gbq.context.credentials = os.environ['GOOGLE_APPLICATION_CREDENTIALS']
pandas_gbq.context.project = "appgpt-374716"

# OpenAI
openai.api_key = config['API_KEYS']['openai']

In [3]:
# Run a select query on the bigquery dataset
def run_query(query):
    query_job = client.query(query)
    results = query_job.result()
    return results

In [8]:
# We receive the following WhatsApp message from the webhook

whatsapp_test = '''
{
    "object": "whatsapp_business_account",
    "entry": [
        {
            "id": "106430019013275",
            "changes": [
                {
                    "value": {
                        "messaging_product": "whatsapp",
                        "metadata": {
                            "display_phone_number": "15555555555",
                            "phone_number_id": "116414838004688"
                        },
                        "contacts": [
                            {
                                "profile": {
                                    "name": "John Doe"
                                },
                                "wa_id": "32123456789"
                            }
                        ],
                        "messages": [
                            {
                                "from": "32123456789",
                                "id": "wamid.HBgLMzI0Nzk0Njc1MzYVAgASGBQzQTQxNTIwMUREOTQzRjEzNzdCOQA=",
                                "timestamp": "1673786869",
                                "text": {
                                    "body": "Hi, What kind of fabrics do you offer ?"
                                },
                                "type": "text"
                            }
                        ]
                    },
                    "field": "messages"
                }
            ]
        }
    ]
}
'''
whatsapp_test = json.loads(whatsapp_test)


In [10]:
# Let's check BigQuery to see if we have a conversation with this user

sender_phone_number = whatsapp_test['entry'][0]['changes'][0]['value']['messages'][0]['from']

query = '''
select WHATSAP_MESSAGE, GPT_RESPONSE, INSERT_TS
from `conversations.chat_history`
where (PHONE_NUMBER = %s) -- phone number
AND (INSERT_TS > DATE_SUB(CURRENT_TIMESTAMP(), INTERVAL 24 HOUR)) -- Only look at the last 24 hours
ORDER BY INSERT_TS DESC
'''
results = pandas_gbq.read_gbq(query % sender_phone_number, credentials=credentials)

print(f'We feteched {len(results)} rows from BigQuery')

Downloading: 100%|██████████|
We feteched 1 rows from BigQuery


/opt/homebrew/Caskroom/miniforge/base/envs/appgpt/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:266: UserWarning: Unable to determine type for field 'WHATSAP_MESSAGE'.
  warnings.warn("Unable to determine type for field '{}'.".format(bq_field.name))


In [11]:
# Initialize a method to answer the question using GPT-3

def answer_question(question):
    global context

    response = openai.Completion.create(
    model="text-davinci-003",
    prompt= context + question,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
    )

    return response


In [12]:
context = "Pretend you are a customer service agent working for a textile wholesale named Quality Textiles.\n\n"
message_id = whatsapp_test['entry'][0]['changes'][0]['value']['messages'][0]['id']
sender_phone_number = whatsapp_test['entry'][0]['changes'][0]['value']['messages'][0]['from']

query = '''
INSERT INTO conversations.chat_history (WHATSAPP_MESSAGE_ID, WHATSAP_MESSAGE, GPT_RESPONSE, PHONE_NUMBER, INSERT_TS)
VALUES (%s, %s, %s, %s, CURRENT_TIMESTAMP())
'''    

# If the result is 0 start a new conversation and set the context
if len(results) == 0:
    #print('No conversation found, starting a new one\n')
    context += whatsapp_test['entry'][0]['changes'][0]['value']['messages'][0]['text']['body']
    #print(context)

    response = json.dumps(answer_question(context))
    # escape all \n characters in the response
    response = re.sub(r'\\n', r'\\\\n', response)

    # Insert the prompt and response into BigQuery
    results = run_query(query % ("'" + message_id + "'", "JSON'" + json.dumps(whatsapp_test) + "'","JSON'" + (response) + "'",sender_phone_number))
    
    print('\n New conversation inserted into BigQuery')

else :
    # Unpack the results
    conversation = context
    for index, row in results.iterrows():
        conversation += f"Q: {json.loads(row['WHATSAP_MESSAGE'])['entry'][0]['changes'][0]['value']['messages'][0]['text']['body']}"
        conversation += f"A: {json.loads(row['GPT_RESPONSE'])['choices'][0]['text']}\n\n"
        
    conversation += f"Q: {whatsapp_test['entry'][0]['changes'][0]['value']['messages'][0]['text']['body']}"
    response = json.dumps(answer_question(conversation))
    
    # escape all \n characters in the response
    response = re.sub(r'\\n', r'\\\\n', response)
    results =  run_query(query % ("'" + message_id + "'", "JSON'" + json.dumps(whatsapp_test) + "'","JSON'" + (response) + "'",sender_phone_number))

    print(conversation)
    print('\n Conversation appended to BigQuery')



    

Pretend you are a customer service agent working for a textile wholesale named Quality Textiles.

Q: Hi, What kind of fabrics do you offer ?

Great question! We offer a wide range of fabrics at Quality Textiles. We specialize in cotton, linen, polyester, silk, wool, and blends. We also offer specialty fabrics such as faux fur, faux leather, vinyl, and upholstery fabrics. We also have a wide selection of prints and patterns for all of our fabrics.

Q: Hi, What kind of fabrics do you offer ?

 Conversation appended to BigQuery
